In [1]:
import sys
# print(sys.path)
sys.path.append('/home/sz/pa2023/')
print(sys.path)

['/home/sz/pa2023/test', '/usr/local/lib/python310.zip', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '', '/home/sz/pa2023/venv/lib/python3.10/site-packages', '/home/sz/pa2023/']


In [15]:
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.experimental import BabyAGI
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, LLMChain
from dotenv import load_dotenv
load_dotenv()
_todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
_todo_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=_todo_prompt
)
from module.tools import tools_faiss_azure_googleserp
_tools = tools_faiss_azure_googleserp
_tool_todo = Tool(
    name="TODO",
    func=_todo_chain.run,
    description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
)
_tools.append(_tool_todo)

_prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks:    {context}."""
_suffix = """Question: {task}
{agent_scratchpad}"""
_prompt = ZeroShotAgent.create_prompt(
    _tools,
    prefix=_prefix,
    suffix=_suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)
_llm = OpenAI(temperature=0)
_llm_chain = LLMChain(
    llm=_llm,
    prompt=_prompt
)
_tool_names = [tool.name for tool in _tools]
_agent = ZeroShotAgent(
    llm_chain=_llm_chain,
    allowed_tools=_tool_names
)
_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=_agent,
    tools=_tools,
    verbose=True
)
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
_vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    _babyagi = BabyAGI.from_llm(
        llm=_llm,
        vectorstore=_vectorstore,
        task_execution_chain=_agent_executor,
        max_iterations=3,
        verbose=False,
    )
    _task = "Write a weather report for SF today"
    _re = _babyagi({"objective": _task})
    _token_cost = f"Tokens: {cb.total_tokens} = (Prompt {cb.prompt_tokens} + Completion {cb.completion_tokens}) Cost: ${format(cb.total_cost, '.5f')}"
    print(_token_cost)
    print(_re)

/home/sz/pa2023/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new  chain...
Thought: I need to create a todo list
Action: TODO
Action Input: Write a weather report for SF today
Observation: 

1. Research current weather conditions in San Francisco
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Analyze data to determine current weather trends
4. Write a brief introduction to the weather report
5. Describe current weather conditions in San Francisco
6. Discuss any upcoming weather changes
7. Summarize the weather report
8. Proofread and edit the report
9. Submit the report
Thought: I now know the final answer
Final Answer: A todo list for writing a weather report for SF today is: 
1. Research current weather conditions in San Francisco
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Analyze data to determine current weather trends
4. Write a brief introduction t